# Create a 1D signal GAN, record from MIC and create recreate the signal
## Use a very basic GAN architecture that works with a single dimension (1D array)

In [1]:
import sounddevice as sd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Audio
from scipy.io.wavfile import write
import os

# Lets setup some basic configurations

In [2]:
sample_rate = 22050
duration = 2  # 2 seconds
time_steps = sample_rate * duration  # 44100 for 2 seconds

## Define the Generator class

In [3]:
# Define the generator
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        output = self.fc(out)  # Take the last output
        return output


## Define the Discriminator class

In [4]:
# Define the discriminator
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.rnn(x)
        output = self.fc(out[:, -1, :])  # Take the last output
        output = self.sigmoid(output)
        return output

## Custom dataset class for voice signals

In [5]:
class AudioDataset(Dataset):
    def __init__(self, data_dir, sample_rate=22050, duration=2):
        self.data_dir = data_dir
        self.sample_rate = sample_rate
        self.duration = duration
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.npy')]  # Only .npy files

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        # Load numpy array
        waveform = np.load(file_path)
        # Ensure the audio is 2 seconds long
        target_length = self.sample_rate * self.duration
        if waveform.size < target_length:
            # Pad the waveform if it's shorter than 2 seconds
            waveform = np.pad(waveform, (0, target_length - waveform.size), mode='constant')
        else:
            # Trim the waveform if it's longer than 2 seconds
            waveform = waveform[:target_length]
        return torch.tensor(waveform, dtype=torch.float32)  # Convert to tensor


## Prepare the dataset for training the GAN

In [6]:
# Create the dataset and dataloader
dataset = AudioDataset('audios/', sample_rate, duration)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
print(len(dataset))

69


## Initialize the generator and discriminator

In [7]:
# Set up the GAN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Instantiate the models
generator = Generator(1, 50, 1)
discriminator = Discriminator(1, 50, 1)

## Define the loss function and optimizers

In [8]:
criterion = nn.BCELoss()
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.001)
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.001)

num_epochs = 10
batch_size = 8

## Training the GAN

In [ ]:
# Training loop
losses = {"g_los}
for epoch in range(num_epochs):
    for batch in dataloader: # batch is (batch_size, time_steps, features)
        # Generate fake signal
        noise = torch.randn(batch.shape[0], batch.shape[1], 1) 
        generated_signal = generator(noise)

        # Create labels
        real_labels = torch.ones(batch.size(0), 1)
        fake_labels = torch.zeros(batch.size(0), 1)

        # Train the discriminator
        discriminator.zero_grad()
        real_output = discriminator(batch)  # Use the real audio batch
        real_loss = criterion(real_output, real_labels)
        fake_output = discriminator(generated_signal.detach())  # Use the generated signal
        fake_loss = criterion(fake_output, fake_labels)
        d_loss = real_loss + fake_loss
        d_loss.backward()
        d_optimizer.step()

        # Train the generator
        generator.zero_grad()
        fake_output = discriminator(generated_signal.detach())  # Use the generated signal
        g_loss = criterion(fake_output, real_labels)
        g_loss.backward()
        g_optimizer.step()

    #if epoch % 100 == 0:
        print(f'Epoch: {epoch}, Generator Loss: {g_loss.item()}, Discriminator Loss: {d_loss.item()}')

Epoch: 0, Generator Loss: 0.7618959546089172, Discriminator Loss: 1.380137324333191
Epoch: 0, Generator Loss: 0.75767582654953, Discriminator Loss: 1.3793529272079468
Epoch: 0, Generator Loss: 0.7531872391700745, Discriminator Loss: 1.3854106664657593
Epoch: 0, Generator Loss: 0.747001051902771, Discriminator Loss: 1.3870664834976196
Epoch: 0, Generator Loss: 0.7362841367721558, Discriminator Loss: 1.3885767459869385
Epoch: 0, Generator Loss: 0.7286736965179443, Discriminator Loss: 1.3865467309951782
Epoch: 0, Generator Loss: 0.7211640477180481, Discriminator Loss: 1.3943531513214111
Epoch: 0, Generator Loss: 0.7107903361320496, Discriminator Loss: 1.3843035697937012
Epoch: 0, Generator Loss: 0.7000789642333984, Discriminator Loss: 1.3845692873001099
Epoch: 1, Generator Loss: 0.6907093524932861, Discriminator Loss: 1.383682131767273
Epoch: 1, Generator Loss: 0.681965172290802, Discriminator Loss: 1.384065866470337
Epoch: 1, Generator Loss: 0.6770755052566528, Discriminator Loss: 1.3814

## Generate new signals after training

In [ ]:
# Set the generator to evaluation mode
generator.eval()
# Generate and visualize a sample output
with torch.no_grad():
    sample_noise = torch.randn(1,time_steps, 1)  # Adjust noise shape
    generated_audio = generator(sample_noise).detach().numpy().flatten()
print(generated_audio.shape)
# Plot the generated audio signal
plt.figure(figsize=(12, 6))
plt.plot(generated_audio, label='Generated Audio Signal', alpha=0.5)
plt.title('Generated Audio Signal')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.legend()
plt.show()